In [93]:
import os, random, glob, pickle, collections, math
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

In [103]:
TRAIN_DIR = '../data/train/'
CHECKPOINT_DIR = './checkpoints/checkpoint4/'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BatchSize = 128
LearningRate = 1e-4


In [72]:
GT_crop_bboxs_df[4370:4372]

,image_class,image_file,crop_index,crop_class,x_min,y_min,x_max,ymax
4370,YFT,img_07911.jpg,1.0,YFT,724.290780,94.680851,1168.617021,347.872340
4371,ALB,img_00003.jpg,0.0,NoF,299.452861,26.285698,1168.160268,286.172768


In [109]:
def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

nbr_perClass = int(BatchSize / len(FISH_CLASSES))   
def generator(datagen = train_datagen):
    while 1:
        batch_x = np.zeros((BatchSize, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BatchSize, len(FISH_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nbr_perClass, replace=False),:]
        batch_df = GT_crop_bboxs_df.groupby('crop_class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(row[0], row[1])
            fish = row[3]
            bbox = row[4:8]
            img = image.load_img(TRAIN_DIR+image_file,target_size=(ROWS,COLS))
            x = img_to_array(img, dim_ordering=self.dim_ordering)
            x = train_datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,FISH_CLASSES.index(fish)] = 1
            i += 1
        yield (batch_x, batch_y)

In [110]:
#Resnet50
#stg1 training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
#x = Dropout(0.5)(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=LearningRate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(generator(), samples_per_epoch=BatchSize*2, nb_epoch=2)

Epoch 1/2


Exception: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.5/site-packages/keras/engine/training.py", line 404, in data_generator_task
    generator_output = next(generator)
  File "<ipython-input-109-66e8a5aeabad>", line 33, in generator
    x = img_to_array(img, dim_ordering=self.dim_ordering)
NameError: name 'img_to_array' is not defined

